# AGVCH Checker

## Get Official Data

In [103]:
import pandas as pd
from datetime import datetime

official_raw = pd.read_csv(f"https://www.agvchapp.bfs.admin.ch/api/communes/snapshot?date={datetime.now().strftime('%d-%m-%Y')}")

# set datatypes
official_raw.Parent = official_raw.Parent.astype("Int64")
official_raw.Inscription = official_raw.Inscription.astype("Int64")
official_raw.BfsCode = official_raw.BfsCode.astype("Int64")

print(official_raw.dtypes)
official_raw.head()

HistoricalCode      int64
BfsCode             Int64
ValidFrom          object
ValidTo           float64
Level               int64
Parent              Int64
Name               object
ShortName          object
Inscription         Int64
Radiation         float64
Rec_Type_fr        object
Rec_Type_de        object
dtype: object


,HistoricalCode,BfsCode,ValidFrom,ValidTo,Level,Parent,Name,ShortName,Inscription,Radiation,Rec_Type_fr,Rec_Type_de
0,1,1,12.09.1848,NaN,1,<NA>,Zürich,ZH,<NA>,NaN,NaN,NaN
1,10053,101,12.09.1848,NaN,2,1,Bezirk Affoltern,Affoltern,100,NaN,NaN,NaN
2,10575,13,12.09.1848,NaN,3,10053,Stallikon,Stallikon,<NA>,NaN,NaN,NaN
3,11742,2,12.09.1848,NaN,3,10053,Affoltern am Albis,Affoltern am Albis,<NA>,NaN,NaN,NaN
4,11801,3,12.09.1848,NaN,3,10053,Bonstetten,Bonstetten,<NA>,NaN,NaN,NaN


## Wrangle Official Data

In [104]:
official = official_raw[official_raw["Level"] == 3]
official = official[["BfsCode", "Name"]]
print(official.dtypes)
official.head()

BfsCode     Int64
Name       object
dtype: object


,BfsCode,Name
2,13,Stallikon
3,2,Affoltern am Albis
4,3,Bonstetten
5,4,Hausen am Albis
6,5,Hedingen


## Wikidata Data

### Get Wikidata Data

In [105]:
import requests

# Endpoint URL
url = "https://query.wikidata.org/sparql"

# Query string to get BFS codes from Wikidata
query = """
SELECT * WHERE {
  ?muni wdt:P31 wd:Q70208.
  OPTIONAL { ?muni wdt:P1448 ?name . }
  OPTIONAL { ?muni wdt:P771 ?bfs . }
  MINUS{ ?muni wdt:P31 wd:Q685309. }
}
"""
# Make the request to the Wikidata SPARQL endpoint
response = requests.get(url, params={'query': query, 'format': 'json'})
wiki_raw = response.json()
wiki_raw = pd.json_normalize(wiki_raw['results']['bindings'])

print(wiki_raw.dtypes)
wiki_raw.head()

muni.type        object
muni.value       object
name.xml:lang    object
name.type        object
name.value       object
bfs.type         object
bfs.value        object
dtype: object


,muni.type,muni.value,name.xml:lang,name.type,name.value,bfs.type,bfs.value
0,uri,http://www.wikidata.org/entity/Q67209,de-ch,literal,Teufen (AR),literal,3024
1,uri,http://www.wikidata.org/entity/Q67215,de-ch,literal,Messen,literal,2457
2,uri,http://www.wikidata.org/entity/Q67213,de-ch,literal,Marthalen,literal,0035
3,uri,http://www.wikidata.org/entity/Q67223,de-ch,literal,Wil (ZH),literal,0071
4,uri,http://www.wikidata.org/entity/Q67220,de-ch,literal,Dürnten,literal,0113


### Wrangle Wikidata

In [106]:
wiki = wiki_raw[["muni.value", "name.value", "name.xml:lang", "bfs.value"]].rename(columns={
    "muni.value": "q",
    "name.value": "name",
    "name.xml:lang": "lang",
    "bfs.value": "bfs"})

wiki["q"] = wiki["q"].str.split("/").str[-1]
wiki["bfsInt"] = wiki["bfs"].astype("Int64")
print(wiki.dtypes)
wiki.head()

q         object
name      object
lang      object
bfs       object
bfsInt     Int64
dtype: object


,q,name,lang,bfs,bfsInt
0,Q67209,Teufen (AR),de-ch,3024,3024
1,Q67215,Messen,de-ch,2457,2457
2,Q67213,Marthalen,de-ch,0035,35
3,Q67223,Wil (ZH),de-ch,0071,71
4,Q67220,Dürnten,de-ch,0113,113


## Merge Dataframes

In [107]:
merge = pd.merge(official, wiki, left_on="BfsCode", right_on="bfsInt", how="outer", indicator=True)
merge["_merge"] = merge["_merge"].map({
    "left_only": "official_only",
    "right_only": "wiki_only",
    "both": "both"
})

# new merge columns
merge["_nameOk"] = merge["Name"] == merge["name"]
merge["_bfsOk"] = merge["bfs"] == merge["bfsInt"].astype(str)

# make categorical columns so that the summarizing is consistent (even if there are no False values, count them as zero)
merge["_nameOk"] = pd.Categorical(merge["_nameOk"], categories=[True, False])
merge["_bfsOk"] = pd.Categorical(merge["_bfsOk"], categories=[True, False])

print(merge.dtypes)
merge.head()

BfsCode       Int64
Name         object
q            object
name         object
lang         object
bfs          object
bfsInt        Int64
_merge     category
_nameOk    category
_bfsOk     category
dtype: object


,BfsCode,Name,q,name,lang,bfs,bfsInt,_merge,_nameOk,_bfsOk
0,1,Aeugst am Albis,Q64945,Aeugst am Albis,de-ch,0001,1,both,True,False
1,2,Affoltern am Albis,Q68290,Affoltern am Albis,de-ch,0002,2,both,True,False
2,3,Bonstetten,Q65510,Bonstetten,de-ch,0003,3,both,True,False
3,4,Hausen am Albis,Q65231,Hausen am Albis,de-ch,0004,4,both,True,False
4,5,Hedingen,Q65129,Hedingen,de-ch,0005,5,both,True,False


## Summarise Merge

In [108]:
# Columns to summarize
columns_to_summarize = ["_merge", "_nameOk", "_bfsOk"]

summary = {col: merge[col].value_counts().to_dict() for col in columns_to_summarize}

summary["_bfsIntUnique"] = merge["bfsInt"].is_unique
summary["_BfsCodeUnique"] = merge["BfsCode"].is_unique

summary
summary["_bfsIntNonUniqueValues"] = merge["BfsCode"][merge["BfsCode"].duplicated(keep=False)].unique().tolist()
summary["_bfsIntNonUniqueValues"] = [x if x is not pd.NA else None for x in summary["_bfsIntNonUniqueValues"]]
summary["_BfsCodeNonUniqueValues"] = merge["bfsInt"][merge["bfsInt"].duplicated(keep=False)].unique().tolist()
summary["_BfsCodeNonUniqueValues"] = [x if x is not pd.NA else None for x in summary["_BfsCodeNonUniqueValues"]]
print(summary)

{'_merge': {'both': 2128, 'wiki_only': 25, 'official_only': 0}, '_nameOk': {True: 1988, False: 165}, '_bfsOk': {True: 1652, False: 501}, '_bfsIntUnique': False, '_BfsCodeUnique': False, '_bfsIntNonUniqueValues': [None, 351, 782, 1331, 2196, 2275, 3291, 3408, 6266, 6289, 6630, 6634, 6644, 6748], '_BfsCodeNonUniqueValues': [351, 782, 1331, 2196, 2275, 3291, 3408, 6266, 6289, 6630, 6634, 6644, 6748, None]}


## Missing LINDAS Versions

### Fetch LINDAS Data

In [109]:
# Endpoint URL
url = "https://ld.admin.ch/query"

headers = {
    "Accept": "application/sparql-results+json"
}

# Query string to get BFS codes from Wikidata
query = """
PREFIX schema: <http://schema.org/>
PREFIX vl: <https://version.link/>
SELECT ?identifier WHERE {
?version a vl:Version;
        vl:inVersionedIdentitySet <https://ld.admin.ch/fso/register>;
        schema:identifier ?identifier.
}
"""
# Make the request to the Wikidata SPARQL endpoint
response = requests.get(url, params={'query': query}, headers=headers)
lindas_raw = response.json()
lindas_raw = pd.json_normalize(lindas_raw['results']['bindings'])

# Convert BFS codes to int
lindas_raw["identifier.value"] = lindas_raw["identifier.value"].astype("Int64")

print(lindas_raw.dtypes)
lindas_raw.head()

identifier.datatype    object
identifier.type        object
identifier.value        Int64
dtype: object


,identifier.datatype,identifier.type,identifier.value
0,http://www.w3.org/2001/XMLSchema#integer,literal,13688
1,http://www.w3.org/2001/XMLSchema#integer,literal,10242
2,http://www.w3.org/2001/XMLSchema#integer,literal,10323
3,http://www.w3.org/2001/XMLSchema#integer,literal,14411
4,http://www.w3.org/2001/XMLSchema#integer,literal,10459


### Search Missing

In [110]:
official_l3 = official_raw[official_raw["Level"] == 3]

missing = official_l3[~official_l3["HistoricalCode"].isin(lindas_raw["identifier.value"])]

# Ensure missing values are handled correctly
missing = missing.replace({pd.NA: None, float('nan'): None})

missing = missing[["HistoricalCode", "BfsCode", "ValidFrom", "Name"]]
print(missing.dtypes)
missing.head()

HistoricalCode     int64
BfsCode            Int64
ValidFrom         object
Name              object
dtype: object


,HistoricalCode,BfsCode,ValidFrom,Name


## Write Files

In [111]:
import json
from datetime import date

# String for today's date
today_str = date.today().isoformat()  # e.g., '2025-08-04'

# merge CSV
merge.to_csv(f"current_munies/{today_str}.csv", index=False)

# Summary JSON
output = {"current_munies": summary,
          "missing_lindas_versions": missing.to_dict(orient="records")}

with open(f"summaries/{today_str}.json", "w", encoding="utf-8") as f:
    json.dump(output, f, indent=4)